In [32]:
import os, sys
import django

sys.path.append('../..')  # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.apps import apps as django_apps

django.setup()
from flourish_caregiver.models.signals import get_child_consents


In [34]:
from flourish_caregiver.models import (TbInformedConsent, 
                                       OnScheduleCohortATb2Months,
                                       OnScheduleCohortATb6Months)

from edc_appointment.models import Appointment
from edc_visit_schedule.models import SubjectScheduleHistory

tb_pids = TbInformedConsent.objects.values_list('subject_identifier', flat=True)

for pid in tb_pids:
    
    child_consents = get_child_consents(pid)
    preg_child_consents = child_consents.filter(preg_enroll=True)
    
    caregiver_visit_count = None
    
    try:
        caregiver_visit_count = preg_child_consents[0].caregiver_visit_count or 1
    except IndexError:
        caregiver_visit_count = 1
    finally:
        
        tb_2_months_schedule_name = f'a_tb{caregiver_visit_count}_2_months_schedule1'
        tb_6_months_schedule_name = f'a_tb{caregiver_visit_count}_6_months_schedule1'
        visit_schedule_name = f'a{caregiver_visit_count}_visit_schedule1'
        
        OnScheduleCohortATb2Months.objects.filter(
            subject_identifier = pid,).update(
            schedule_name=tb_2_months_schedule_name)
        
        OnScheduleCohortATb6Months.objects.filter(
            subject_identifier=pid).update(
            schedule_name=tb_6_months_schedule_name
        )
        
        SubjectScheduleHistory.objects.filter(
            subject_identifier = pid, 
            schedule_name='tb_2_months_schedule').update(
            schedule_name = tb_2_months_schedule_name,
            visit_schedule_name = visit_schedule_name
        )
        
        SubjectScheduleHistory.objects.filter(
            subject_identifier = pid, 
            schedule_name='tb_6_months_schedule').update(
            schedule_name = tb_6_months_schedule_name,
            visit_schedule_name = visit_schedule_name
        )
        
        Appointment.objects.filter(
            schedule_name = 'tb_2_months_schedule',
            subject_identifier = pid
        ).update(
            visit_schedule_name = visit_schedule_name,
            schedule_name=tb_2_months_schedule_name)
        
        Appointment.objects.filter(
            schedule_name = 'tb_6_months_schedule',
            subject_identifier= pid
        ).update(
            visit_schedule_name = visit_schedule_name,
            schedule_name=tb_6_months_schedule_name)


